In [ ]:
!pip install nibabel
!pip install seaborn
!pip install openpyxl

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Input
from keras.layers import Convolution3D,Conv3D,Add,concatenate, MaxPool3D,add, Convolution2D,BatchNormalization, GlobalAveragePooling3D, ZeroPadding3D
from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, MaxPool3D, Flatten, Dense, Dropout, concatenate, Lambda, Layer
from keras import backend as K
import cv2
import operator
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
import nibabel as nib
import pandas as pd
from scipy.ndimage import rotate
import random
import nibabel as nib
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import nibabel as nib
from matplotlib import animation, rc
from scipy.ndimage import rotate
import glob
import os
from sklearn.model_selection import KFold, StratifiedKFold,train_test_split
from scipy.ndimage import zoom
from matplotlib.patches import PathPatch, Rectangle
from IPython.display import HTML
from tensorflow.keras import layers

from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, MaxPool3D, Flatten, Dense, Dropout, concatenate, Lambda, Layer
from tensorflow.keras.models import Model
import tensorflow as tf

In [ ]:
# Enable TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Device:", tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print("Number of accelerators: ", strategy.num_replicas_in_sync)

### FUNCTIONS

In [ ]:
def define_model():
    model = Sequential()
    model.add(Conv3D(32,(3,3,3), activation='relu', kernel_initializer='he_uniform', padding='same',input_shape=(32,32,32,1)))
    model.add(BatchNormalization())
    model.add(Conv3D(32, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    #model.add(Dropout(0.1))
    
    model.add(Conv3D(64, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(64, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    #model.add(Dropout(0.2))
    
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))
    model.add(Dropout(0.3))
    
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(128, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPool3D((2, 2,2)))

    model.add(Conv3D(256, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv3D(256, (3, 3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(BatchNormalization())
    #model.add(MaxPool3D((2, 2,2))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    #model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Dense(8, activation='relu', kernel_initializer='he_uniform'))


    model.add(Dense(1, activation='sigmoid', kernel_initializer='he_uniform'));
    return model

In [ ]:
BS=68 # batch size
RS= 0 #random state
E = 200 #epoch

In [ ]:
def aug_rotate(x):

    rotated_slices = []
    for slice_idx in range(x.shape[2]):
        slice_data = x[:, :, slice_idx]  # Extract a single slice
        rotated_slice = rotate(slice_data, angle=angles[j], reshape=False, mode='nearest')
        rotated_slices.append(rotated_slice)

    # Create a new NIfTI image from the rotated slices
    rotated_img_data = np.stack(rotated_slices, axis=-1)
    # Create a new NIfTI image using the header information from the original image
    rotated_nifti = nib.Nifti1Image(rotated_img_data, img.affine)
    
    return rotated_nifti

### DATA LOAD

In [ ]:
luna_df = pd.read_excel('/kaggle/input/spie-cropped-resampled-dataset/final_Luna16.xlsx')
column_luna = 'malignancy'
luna_series=luna_df['Series Uid'].tolist()

luna_folder = '/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled'  
luna = []
for i in luna_series:
    nii_file_path = os.path.join(luna_folder, f'{i}.nii')  # Assuming NII files have the extension '.nii'
    
    if os.path.exists(nii_file_path):
        luna.append(i)
luna=np.array(luna)

print(luna.shape)

### DATA PREPARATION FOR SSL(UNLABELED)

In [ ]:
luna_semi_df = pd.read_excel('/kaggle/input/spie-cropped-resampled-dataset/unlabeled_data_Luna.xlsx')
#column_name = 'malignancy'
col_scan='Series Uid'

luna_semi_scan = luna_semi_df[col_scan].tolist()

scans1=[]
for i in luna_semi_scan:
    scans1.append(i)

luna_semi=np.array(scans1)
print(len(luna_semi))

In [ ]:
datas_luna_semi=[]
scans_luna_semi=[]

for i in luna_semi:
    img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/",i +"*.nii"))

    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_luna_semi.append(data)
        scans_luna_semi.append(i)

X_luna_semi = np.array(datas_luna_semi)
z_luna_semi= np.array(scans_luna_semi)

print(X_luna_semi.shape)

### FIXED TEST SET

In [ ]:
luna_initial,luna_fixed_test = train_test_split(luna, test_size=0.2, random_state=RS)

z_luna_fixed_test=np.array(luna_fixed_test)
z_luna_initial=np.array(luna_initial)

print(luna_initial.shape)# initial train
print(luna_fixed_test.shape)

In [ ]:
luna_fixed_test_df = pd.DataFrame({'luna fixed test set': luna_fixed_test})
luna_initial_df = pd.DataFrame({'luna initial set': luna_initial})


# Create a new DataFrame with the matched scans
matched_df = pd.DataFrame({'luna fixed test set': luna_fixed_test_df['luna fixed test set'],
                           'luna initial set': luna_initial_df['luna initial set'],})



matched_df.to_excel('/kaggle/working/scans_for_semisupervised.xlsx', index=False)

In [ ]:
datas_luna_fixed_test=[]
labels_luna_fixed_test=[]
scans_luna_fixed_test=[]

for i in luna_fixed_test:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/",i +"*.nii"))

    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_luna_fixed_test.append(data)
        labels_luna_fixed_test.append(lab)
        scans_luna_fixed_test.append(i)

X_luna_fixed_test = np.array(datas_luna_fixed_test)
y_luna_fixed_test = np.array(labels_luna_fixed_test)
z_luna_fixed_test= np.array(scans_luna_fixed_test)

print(X_luna_fixed_test.shape)

###  FIRST TRAIN

In [ ]:
z_train1, z_val1 = train_test_split(z_luna_initial, test_size=0.2, random_state=RS)
print(z_train1.shape,z_val1.shape)

In [ ]:
angles = [0, 45, 90, 135, 180, 225, 270, 315]


datas_train = []
labels_train=[]

datas_val = []
labels_val=[]

for i in z_train1:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/",i +"*.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        
        for j in range(len(angles)):
            rotated_nifti = aug_rotate(data)
            datas_train.append(rotated_nifti.get_fdata())
            labels_train.append(lab)
            
            
for i in z_val1:
    lab = luna_df.loc[luna_df['Series Uid'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/",i +"*.nii"))
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()
        datas_val.append(data)
        labels_val.append(lab)


X_train1=np.array(datas_train)
y_train1=np.array(labels_train)

X_val1=np.array(datas_val)
y_val1=np.array(labels_val)        

print(y_train1.shape,sum(y_train1))
print(y_val1.shape,sum(y_val1))

In [ ]:
with strategy.scope():
    model=define_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('semiSupervised_HU1.h5', monitor='val_accuracy',save_best_only=True)

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync
# Fit data to model
history = model.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[lr_callback,checkpoint_callback],verbose=0)


# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model1 = tf.keras.models.load_model('semiSupervised_HU1.h5')

loss1, accuracy1 = best_model1.evaluate(X_luna_fixed_test, y_luna_fixed_test)   


print(f'Luna Accuracy: {accuracy1}')
print('luna loss: ',loss1)


In [ ]:
predictions = best_model1.predict(X_luna_fixed_test)
predicted_classes = (predictions > 0.5).astype(int)

# Generate the classification report
class_names = list(['benign','malignant'])
report1 = classification_report(y_luna_fixed_test, predicted_classes , target_names=class_names)
print("Classification Report:\n", report1)

# Generate the confusion matrix
cm1 = confusion_matrix(y_luna_fixed_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm1, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

auc = roc_auc_score(y_luna_fixed_test, predictions)
print(f'AUC: {auc}')

#---------------------------------------------------------
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
import pickle
pickle.dump(X_luna_fixed_test,open('test_x','wb'))
pickle.dump(y_luna_fixed_test,open('test_y','wb'))
pickle.dump(z_luna_fixed_test,open('test_z','wb'))


pickle.dump(X_val1,open('val1_x','wb'))
pickle.dump(y_val1,open('val1_y','wb'))
pickle.dump(z_val1,open('val1_z','wb'))



### prediction from unbalanced data

In [ ]:
correct_predictions1 = []
incorrect_predictions1 = []

predictions = best_model1.predict(X_luna_semi)
for i in range(len(X_luna_semi)):
    if predictions[i] > 0.9:
        correct_predictions1.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions1.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions1.append(z_luna_semi[i])

print(len(incorrect_predictions1),len(correct_predictions1))
        
# Create a DataFrame with 'scan' and 'malignancy' columns
df_correct_predictions1 = pd.DataFrame(correct_predictions1, columns=['scan', 'malignancy'])

scan_identifiers_z_train2 = np.array([item[0] for item in correct_predictions1])


In [ ]:
datas_train = []
labels_train=[]
scans_train=[]

datas_val = []
labels_val=[]
scans_val=[]



for i in scan_identifiers_z_train2:
    lab = df_correct_predictions1.loc[df_correct_predictions1['scan'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

    scans_train.append(i)
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()

        for j in range(len(angles)):
            rotated_nifti = aug_rotate(data)
            datas_train.append(rotated_nifti.get_fdata())
            labels_train.append(lab)



X_train2=np.array(datas_train)
y_train2=np.array(labels_train)
z_train2=np.array(scans_train)

print(z_train2.shape)

In [ ]:
X_train = np.concatenate((X_train1, X_train2))
y_train = np.concatenate((y_train1, y_train2))
z_train = np.concatenate((z_train1, z_train2))
print(z_train.shape)

## second train

In [ ]:
with strategy.scope():
    model=define_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('semiSupervised_HU2.h5', monitor='val_accuracy',save_best_only=True)

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync
# Fit data to model
history = model.fit(X_train, y_train, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[lr_callback,checkpoint_callback],verbose=0)


# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model2 = tf.keras.models.load_model('semiSupervised_HU2.h5')

loss2, accuracy2 = best_model2.evaluate(X_luna_fixed_test, y_luna_fixed_test)   


print(f'Luna Accuracy: {accuracy2}')
print('luna loss: ',loss2)



In [ ]:
predictions = best_model2.predict(X_luna_fixed_test)
predicted_classes = (predictions > 0.5).astype(int)

# Generate the classification report
class_names = list(['benign','malignant'])
report2 = classification_report(y_luna_fixed_test, predicted_classes , target_names=class_names)
print("Classification Report:\n", report2)

# Generate the confusion matrix
cm2 = confusion_matrix(y_luna_fixed_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm2, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

auc = roc_auc_score(y_luna_fixed_test, predictions)
print(f'AUC: {auc}')
#---------------------------------------------------------
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

### unpredicted data after 2nd train

In [ ]:
datas_semi = []
labels_semi = []
scans_semi = []

for i in incorrect_predictions1:
        img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            datas_semi.append(data)
            scans_semi.append(i)

X_semi2 = np.array(datas_semi)
z_semi2 = np.array(scans_semi)

print(X_semi2.shape)


In [ ]:
correct_predictions2 = []
incorrect_predictions2 = []

predictions = best_model2.predict(X_semi2)
for i in range(len(X_semi2)):
    if predictions[i] > 0.9:
        correct_predictions2.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions2.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions2.append(z_luna_semi[i])

print(len(incorrect_predictions2),len(correct_predictions2))

# Create a DataFrame with 'scan' and 'malignancy' columns
df_correct_predictions2 = pd.DataFrame(correct_predictions2, columns=['scan', 'malignancy'])


scan_identifiers_z_train3 = np.array([item[0] for item in correct_predictions2])

In [ ]:
datas_train = []
labels_train=[]
scans_train=[]

datas_val = []
labels_val=[]
scans_val=[]



for i in scan_identifiers_z_train3:
    lab = df_correct_predictions2.loc[df_correct_predictions2['scan'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

    scans_train.append(i)
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()

        for j in range(len(angles)):
            rotated_nifti = aug_rotate(data)
            datas_train.append(rotated_nifti.get_fdata())
            labels_train.append(lab)



X_train3=np.array(datas_train)
y_train3=np.array(labels_train)
z_train3=np.array(scans_train)

print(z_train3.shape)

In [ ]:
X_train = np.concatenate((X_train1, X_train2 , X_train3))
y_train = np.concatenate((y_train1, y_train2 , y_train3))
z_train = np.concatenate((z_train1, z_train2 , z_train3))
print(z_train.shape)

## 3rd train

In [ ]:
with strategy.scope():
    model=define_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('semiSupervised_HU3.h5', monitor='val_accuracy',save_best_only=True)

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync
# Fit data to model
history = model.fit(X_train, y_train, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[lr_callback,checkpoint_callback],verbose=0)


# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model3 = tf.keras.models.load_model('semiSupervised_HU3.h5')

loss3, accuracy3 = best_model3.evaluate(X_luna_fixed_test, y_luna_fixed_test)   

print(f'Luna Accuracy: {accuracy3}')
print('luna loss: ',loss3)



In [ ]:
predictions = best_model3.predict(X_luna_fixed_test)
predicted_classes = (predictions > 0.5).astype(int)

# Generate the classification report
class_names = list(['benign','malignant'])
report3 = classification_report(y_luna_fixed_test, predicted_classes , target_names=class_names)
print("Classification Report:\n", report3)

# Generate the confusion matrix
cm3 = confusion_matrix(y_luna_fixed_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm3, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

auc = roc_auc_score(y_luna_fixed_test, predictions)
print(f'AUC: {auc}')
#---------------------------------------------------------
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

### unpredicted data after 3rd train

In [ ]:
datas_semi = []
labels_semi = []
scans_semi = []

for i in incorrect_predictions2:
        img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            datas_semi.append(data)
            scans_semi.append(i)

X_semi3 = np.array(datas_semi)
z_semi3 = np.array(scans_semi)

print(X_semi3.shape)

In [ ]:
correct_predictions3 = []
incorrect_predictions3 = []

predictions = best_model3.predict(X_semi3)
for i in range(len(X_semi3)):
    if predictions[i] > 0.9:
        correct_predictions3.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions3.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions3.append(z_luna_semi[i])

print(len(incorrect_predictions3),len(correct_predictions3))
        
# Create a DataFrame with 'scan' and 'malignancy' columns
df_correct_predictions3 = pd.DataFrame(correct_predictions3, columns=['scan', 'malignancy'])

print(df_correct_predictions3.shape)


scan_identifiers_z_train4 = np.array([item[0] for item in correct_predictions3])

In [ ]:
datas_train = []
labels_train=[]
scans_train=[]

datas_val = []
labels_val=[]
scans_val=[]



for i in scan_identifiers_z_train4:
    lab = df_correct_predictions3.loc[df_correct_predictions3['scan'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

    scans_train.append(i)
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()

        for j in range(len(angles)):
            rotated_nifti = aug_rotate(data)
            datas_train.append(rotated_nifti.get_fdata())
            labels_train.append(lab)



X_train4=np.array(datas_train)
y_train4=np.array(labels_train)
z_train4=np.array(scans_train)

print(z_train4.shape)

In [ ]:
X_train = np.concatenate((X_train1, X_train2 , X_train3 , X_train4))
y_train = np.concatenate((y_train1, y_train2 , y_train3 , y_train4))
z_train = np.concatenate((z_train1, z_train2 , z_train3 , z_train4))
print(z_train.shape)

## 4th train

In [ ]:
with strategy.scope():
    model=define_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('semiSupervised_HU4.h5', monitor='val_accuracy',save_best_only=True)

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync
# Fit data to model
history = model.fit(X_train, y_train, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[lr_callback,checkpoint_callback],verbose=0)


# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model4 = tf.keras.models.load_model('semiSupervised_HU4.h5')

loss4, accuracy4 = best_model4.evaluate(X_luna_fixed_test, y_luna_fixed_test)   
print(f'Luna Accuracy: {accuracy4}')
print('luna loss: ',loss4)

In [ ]:
predictions = best_model4.predict(X_luna_fixed_test)
predicted_classes = (predictions > 0.5).astype(int)

# Generate the classification report
class_names = list(['benign','malignant'])
report4 = classification_report(y_luna_fixed_test, predicted_classes , target_names=class_names)
print("Classification Report:\n", report4)

# Generate the confusion matrix
cm4 = confusion_matrix(y_luna_fixed_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm4, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

auc = roc_auc_score(y_luna_fixed_test, predictions)
print(f'AUC: {auc}')
#---------------------------------------------------------
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
pickle.dump(X_train4,open('train4_x','wb'))
pickle.dump(y_train4,open('train4_y','wb'))
pickle.dump(z_train4,open('train4_z','wb'))

### unpredicted data after 4th train

In [ ]:
datas_semi = []
labels_semi = []
scans_semi = []

for i in incorrect_predictions3:
        img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            datas_semi.append(data)
            scans_semi.append(i)

X_semi4 = np.array(datas_semi)
z_semi4 = np.array(scans_semi)

print(X_semi4.shape)

In [ ]:
correct_predictions4 = []
incorrect_predictions4 = []

predictions = best_model4.predict(X_semi4)
for i in range(len(X_semi4)):
    if predictions[i] > 0.9:
        correct_predictions4.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions4.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions4.append(z_luna_semi[i])

# Create a DataFrame with 'scan' and 'malignancy' columns
df_correct_predictions4 = pd.DataFrame(correct_predictions4, columns=['scan', 'malignancy'])

print(df_correct_predictions4.shape)
print(len(incorrect_predictions4),len(correct_predictions4))

scan_identifiers_z_train5 = np.array([item[0] for item in correct_predictions4])

In [ ]:
datas_train = []
labels_train=[]
scans_train=[]

datas_val = []
labels_val=[]
scans_val=[]



for i in scan_identifiers_z_train5:
    lab = df_correct_predictions4.loc[df_correct_predictions4['scan'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

    scans_train.append(i)
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()

        for j in range(len(angles)):
            rotated_nifti = aug_rotate(data)
            datas_train.append(rotated_nifti.get_fdata())
            labels_train.append(lab)



X_train5=np.array(datas_train)
y_train5=np.array(labels_train)
z_train5=np.array(scans_train)

print(z_train5.shape)

In [ ]:
X_train = np.concatenate((X_train1, X_train2 , X_train3 , X_train4 , X_train5))
y_train = np.concatenate((y_train1, y_train2 , y_train3 , y_train4 , y_train5))
z_train = np.concatenate((z_train1, z_train2 , z_train3 , z_train4 , z_train5))
print(z_train.shape)

## 5th train

In [ ]:
with strategy.scope():
    model=define_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('semiSupervised_HU5.h5', monitor='val_accuracy',save_best_only=True)

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync
# Fit data to model
history = model.fit(X_train, y_train, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[lr_callback,checkpoint_callback],verbose=0)


# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model5 = tf.keras.models.load_model('semiSupervised_HU5.h5')

loss5, accuracy5 = best_model5.evaluate(X_luna_fixed_test, y_luna_fixed_test)   

print(f'Luna Accuracy: {accuracy5}')
print('luna loss: ',loss5)


In [ ]:
predictions = best_model5.predict(X_luna_fixed_test)
predicted_classes = (predictions > 0.5).astype(int)

# Generate the classification report
class_names = list(['benign','malignant'])
report5 = classification_report(y_luna_fixed_test, predicted_classes , target_names=class_names)
print("Classification Report:\n", report5)

# Generate the confusion matrix
cm5 = confusion_matrix(y_luna_fixed_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm5, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


auc = roc_auc_score(y_luna_fixed_test, predictions)
print(f'AUC: {auc}')
#---------------------------------------------------------
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
pickle.dump(X_train5,open('train5_x','wb'))
pickle.dump(y_train5,open('train5_y','wb'))
pickle.dump(z_train5,open('train5_z','wb'))

### unpredicted data after fifth train

In [ ]:
datas_semi = []
labels_semi = []
scans_semi = []

for i in incorrect_predictions4:
        img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            datas_semi.append(data)
            scans_semi.append(i)

X_semi5 = np.array(datas_semi)
z_semi5 = np.array(scans_semi)

print(X_semi5.shape)

In [ ]:
predictions = best_model5.predict(X_semi5)
correct_predictions5 = []
incorrect_predictions5 = []

for i in range(len(X_semi5)):
    if predictions[i] > 0.9:
        correct_predictions5.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions5.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions5.append(z_luna_semi[i])

# Create a DataFrame with 'scan' and 'malignancy' columns
df_correct_predictions5 = pd.DataFrame(correct_predictions5, columns=['scan', 'malignancy'])

print(len(incorrect_predictions5), len(correct_predictions5))

scan_identifiers_z_train6 = np.array([item[0] for item in correct_predictions5])

In [ ]:
datas_train = []
labels_train=[]
scans_train=[]

datas_val = []
labels_val=[]
scans_val=[]



for i in scan_identifiers_z_train6:
    lab = df_correct_predictions5.loc[df_correct_predictions5['scan'] == i, 'malignancy'].values[0]
    img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

    scans_train.append(i)
    for img_file in img_files:
        img = nib.load(img_file)
        data = img.get_fdata()

        for j in range(len(angles)):
            rotated_nifti = aug_rotate(data)
            datas_train.append(rotated_nifti.get_fdata())
            labels_train.append(lab)



X_train6=np.array(datas_train)
y_train6=np.array(labels_train)
z_train6=np.array(scans_train)

print(z_train6.shape)

In [ ]:
X_train = np.concatenate((X_train1, X_train2 , X_train3 , X_train4 , X_train5 , X_train6))
y_train = np.concatenate((y_train1, y_train2 , y_train3 , y_train4 , y_train5 , y_train6))
z_train = np.concatenate((z_train1, z_train2 , z_train3 , z_train4 , z_train5 , z_train6))
print(z_train.shape)

## 6th train

In [ ]:
with strategy.scope():
    model=define_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)

# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('semiSupervised_HU6.h5', monitor='val_accuracy',save_best_only=True)

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync
# Fit data to model
history = model.fit(X_train, y_train, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[lr_callback,checkpoint_callback],verbose=0)


# Evaluate the model - report accuracy and capture it into a list for future reporting
best_model6 = tf.keras.models.load_model('semiSupervised_HU6.h5')

loss6, accuracy6 = best_model6.evaluate(X_luna_fixed_test, y_luna_fixed_test)   
print(f'Luna Accuracy: {accuracy6}')
print('luna loss: ',loss6)


In [ ]:
predictions = best_model6.predict(X_luna_fixed_test)
predicted_classes = (predictions > 0.5).astype(int)

# Generate the classification report
class_names = list(['benign','malignant'])
report6 = classification_report(y_luna_fixed_test, predicted_classes , target_names=class_names)
print("Classification Report:\n", report6)

auc = roc_auc_score(y_luna_fixed_test, predictions)
print(f'AUC: {auc}')

# Generate the confusion matrix
cm6 = confusion_matrix(y_luna_fixed_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm6, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()



#---------------------------------------------------------
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
pickle.dump(X_train6,open('train5_x','wb'))
pickle.dump(y_train6,open('train5_y','wb'))
pickle.dump(z_train6,open('train5_z','wb'))

### unpredicted data after sixth train

In [ ]:
datas_semi = []
labels_semi = []
scans_semi = []

for i in incorrect_predictions5:
        img_files = glob.glob(os.path.join("/kaggle/input/spie-cropped-resampled-dataset/Luna_16_cropped_resampled/Luna_16_cropped_resampled/", i + "*.nii"))

        for img_file in img_files:
            img = nib.load(img_file)
            data = img.get_fdata()
            datas_semi.append(data)
            scans_semi.append(i)

X_semi6 = np.array(datas_semi)
z_semi6 = np.array(scans_semi)

print(X_semi6.shape)

In [ ]:
predictions = best_model6.predict(X_semi6)
correct_predictions6 = []
incorrect_predictions6 = []

for i in range(len(X_semi6)):
    if predictions[i] > 0.9:
        correct_predictions6.append((z_luna_semi[i], 1))
    elif predictions[i] < 0.1:
        correct_predictions6.append((z_luna_semi[i], 0))
    else:
        incorrect_predictions6.append(z_luna_semi[i])

# Create a DataFrame with 'scan' and 'malignancy' columns
df_correct_predictions6 = pd.DataFrame(correct_predictions6, columns=['scan', 'malignancy'])

print(len(incorrect_predictions6), len(correct_predictions6))

scan_identifiers_z_train7 = np.array([item[0] for item in correct_predictions6])

In [ ]:
import pandas as pd

# Assuming z_train1, z_train2, etc., are numpy arrays
z_train_lists = [z_train1, z_train2, z_train3, z_train4, z_train5, z_train6]
z_val_lists = [z_val1]

# Create DataFrames from numpy arrays with unique column names
df_train_list = [pd.DataFrame({f'z_train{i + 1}': z_train}) for i, z_train in enumerate(z_train_lists)]
df_val_list = [pd.DataFrame({'z_val': z_val}) for z_val in z_val_lists]

# Concatenate the DataFrames along the columns
df_combined = pd.concat(df_val_list + df_train_list, axis=1)

# Save the combined DataFrame to an Excel file
df_combined.to_excel('combined_train_val_lists.xlsx', index=False)


In [ ]:
all_dfs = [df_correct_predictions1, df_correct_predictions2,df_correct_predictions3,df_correct_predictions4,df_correct_predictions5]
result_df = pd.concat(all_dfs, ignore_index=True)

# Save the concatenated DataFrame to an Excel file
result_df.to_excel('output_file.xlsx', index=False)

In [ ]:
accuracy_list = [round(accuracy * 100, 2) for accuracy in [accuracy1, accuracy2,accuracy3, accuracy4, accuracy5, accuracy6]]
loss_list = [round(loss * 100, 2) for loss in [loss1, loss2,loss3, loss4, loss5, loss6]]

df_accloss = pd.DataFrame({ 'accuracy': accuracy_list, 'loss': loss_list})
print(df_accloss)

In [ ]:
X_train = np.concatenate((X_train1, X_train2 , X_train3 , X_train4 , X_train5,X_train6))
y_train = np.concatenate((y_train1, y_train2 , y_train3 , y_train4 , y_train5,y_train6))
z_train = np.concatenate((z_train1, z_train2 , z_train3 , z_train4 , z_train5,z_train6))
print(z_train.shape)

X_test= X_luna_fixed_test
y_test= y_luna_fixed_test
z_test= z_luna_fixed_test

pickle.dump(X_train,open('train_x','wb'))
pickle.dump(y_train,open('train_y','wb'))
pickle.dump(z_train,open('train_z','wb'))

# HU

# Dividing into 2 branches random initializer

In [ ]:
# Custom layer with trainable variables
from tensorflow.keras.constraints import Constraint

class RangeConstraint(Constraint):
    def __init__(self, min_value=0.0, max_value=1.0):
        self.min_value = min_value
        self.max_value = max_value

    def __call__(self, w):
        return tf.clip_by_value(w, self.min_value, self.max_value)

    def get_config(self):
        return {'min_value': self.min_value, 'max_value': self.max_value}


# Custom layer with trainable variables
class LearnableDynamicRange(Layer):
    def __init__(self, **kwargs):
        # Use the provided name or the default name if not provided
        name = kwargs.pop('name', 'learnable_dynamic_range')
        super(LearnableDynamicRange, self).__init__(name=name, **kwargs)

        # Define trainable variables with initial values
        self.a = self.add_weight(
            name='a',
            shape=(1,),
            initializer=tf.random_uniform_initializer(minval=0.0, maxval=1.0),
            constraint=RangeConstraint(min_value=0.3, max_value=0.7),
            trainable=True
        )
      
        
    def call(self, inputs):
        
        a_layer = tf.where(inputs > self.a, 1.0, inputs / self.a)
        b_layer = tf.clip_by_value((inputs - self.a) / (1 - self.a), 0, 1)


        return a_layer, b_layer


# Model
def conv_block(inputs):
    x = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((2, 2, 2))(x)
    
    x = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(64, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((2, 2, 2))(x)
    
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((2, 2, 2))(x)
    x = Dropout(0.3)(x)
    
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((2, 2, 2))(x)
    
    x = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv3D(256, (3, 3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPool3D((2, 2, 2))(x)
    x = Dropout(0.5)(x)
    
    x = Flatten()(x)
    
    return x

def define_model():
    input_shape = (32, 32, 32, 1)
    input_data = Input(shape=input_shape)

    # Use the custom layer with trainable variables
    dynamic_range_layer = LearnableDynamicRange()(input_data)
    a_layer, b_layer= dynamic_range_layer

    branch_a = conv_block(a_layer)
    branch_b = conv_block(b_layer)
    
    concatenated_branches = concatenate([branch_a, branch_b])

    x = Dense(128, activation='relu', kernel_initializer='he_uniform')(concatenated_branches)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(8, activation='relu', kernel_initializer='he_uniform')(x)

    output_layer = Dense(1, activation='sigmoid', kernel_initializer='he_uniform')(x)

    model = Model(inputs=input_data, outputs=output_layer)

    return model

In [ ]:
with strategy.scope():
    model=define_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)
checkpoint_callback = ModelCheckpoint('Luna_train1.h5', monitor='val_accuracy',save_best_only=True, save_format='tf')


BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync


history = model.fit(X_train, y_train, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[checkpoint_callback,lr_callback],verbose=0)


best_model1 = tf.keras.models.load_model('Luna_train1.h5', custom_objects={'LearnableDynamicRange': LearnableDynamicRange})

loss1, accuracy1 = best_model1.evaluate(X_test, y_test)
print('accuracy: ',accuracy1*100)
print('loss: ',loss1*100)

print('-----------------------------------------------------------------')
a_value = best_model1.get_layer('learnable_dynamic_range').a.numpy()
print("Value of 'a' in the best model:", a_value)

In [ ]:
auc_df = pd.DataFrame(columns=['AUC'])   


luna_correct_hu1=[]
predictions1 = best_model1.predict(X_test)
predicted_classes = (predictions1 > 0.5).astype(int)


#---------------------------------------------------------------------------------
# Generate the classification report
class_names = list(['benign','malignant'])
report = classification_report(y_test, predicted_classes, target_names=class_names,digits=5)
print("Classification Report:\n", report)

# Calculate AUC
auc = roc_auc_score(y_test, predictions1)
print(f'AUC: {auc}')

# Generate the confusion matrix
cm = confusion_matrix(y_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# Extract micro avg 
report0 = classification_report(y_test, predicted_classes, target_names=class_names,digits=5,output_dict=True)
precision = report0['macro avg']['precision']
recall = report0['macro avg']['recall']
f1_score = report0['macro avg']['f1-score']
combined_row = {'AUC': auc*100, 'Precision': precision*100, 'Recall': recall*100, 'F1-Score': f1_score*100}
combined_row_df = pd.DataFrame([combined_row])
auc_df = pd.concat([auc_df, combined_row_df], ignore_index=True)

# Dividing into 2 branches constant initializer


In [ ]:
# Custom layer with trainable variables
class LearnableDynamicRange(Layer):
    def __init__(self, initial_a, **kwargs):
        # Use the provided name or the default name if not provided
        name = kwargs.pop('name', 'learnable_dynamic_range')
        super(LearnableDynamicRange, self).__init__(name=name, **kwargs)

        # Define trainable variables with initial values
        self.a = self.add_weight(
            name='a',
            shape=(1,),
            initializer=tf.constant_initializer(value=initial_a),
            trainable=True
        )
       
        
    def call(self, inputs):
        
        a_layer = tf.where(inputs > self.a, 1.0, inputs / self.a)
        b_layer = tf.clip_by_value((inputs - self.a) / (1 - self.a), 0, 1)


        return a_layer, b_layer


def define_model(initial_a):
    input_shape = (32, 32, 32, 1)
    input_data = Input(shape=input_shape)

    # Use the custom layer with trainable variables
    dynamic_range_layer = LearnableDynamicRange(initial_a=initial_a)(input_data)
    a_layer, b_layer = dynamic_range_layer

    branch_a = conv_block(a_layer)
    branch_b = conv_block(b_layer)
    

    concatenated_branches = concatenate([branch_a, branch_b])

    x = Dense(128, activation='relu', kernel_initializer='he_uniform')(concatenated_branches)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(8, activation='relu', kernel_initializer='he_uniform')(x)

    output_layer = Dense(1, activation='sigmoid', kernel_initializer='he_uniform')(x)

    model = Model(inputs=input_data, outputs=output_layer)

    return model

In [ ]:
with strategy.scope():
    model=define_model(0.5)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)

checkpoint_callback = ModelCheckpoint('Luna_train2.h5', monitor='val_accuracy',save_best_only=True, save_format='tf')

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync


history = model.fit(X_train, y_train, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[checkpoint_callback,lr_callback],verbose=0)
    

best_model2 = tf.keras.models.load_model('Luna_train2.h5', custom_objects={'LearnableDynamicRange': LearnableDynamicRange})

loss2, accuracy2 = best_model2.evaluate(X_test, y_test)
print('accuracy: ',accuracy2*100)
print('loss: ',loss2*100)

print('-----------------------------------------------------------------')
a_value = best_model2.get_layer('learnable_dynamic_range').a.numpy()
print("Value of 'a' in the best model:", a_value)

In [ ]:
luna_correct_hu2=[]
predictions2 = best_model2.predict(X_test)
predicted_classes = (predictions2 > 0.5).astype(int)


#---------------------------------------------------------------------------------
# Generate the classification report
class_names = list(['benign','malignant'])
report = classification_report(y_test, predicted_classes, target_names=class_names,digits=5)
print("Classification Report:\n", report)

# Calculate AUC
auc = roc_auc_score(y_test, predictions2)
print(f'AUC: {auc}')



# Generate the confusion matrix
cm = confusion_matrix(y_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# Extract micro avg 
report0 = classification_report(y_test, predicted_classes, target_names=class_names,digits=5,output_dict=True)
precision = report0['macro avg']['precision']
recall = report0['macro avg']['recall']
f1_score = report0['macro avg']['f1-score']
combined_row = {'AUC': auc*100, 'Precision': precision*100, 'Recall': recall*100, 'F1-Score': f1_score*100}
combined_row_df = pd.DataFrame([combined_row])
auc_df = pd.concat([auc_df, combined_row_df], ignore_index=True)

# Dividing into 3 branches constant initializer

In [ ]:
# Custom layer with trainable variables
class LearnableDynamicRange(Layer):
    def __init__(self, initial_a, initial_b, **kwargs):
        # Use the provided name or the default name if not provided
        name = kwargs.pop('name', 'learnable_dynamic_range')
        super(LearnableDynamicRange, self).__init__(name=name, **kwargs)

        # Define trainable variables with initial values
        self.a = self.add_weight(
            name='a',
            shape=(1,),
            initializer=tf.constant_initializer(value=initial_a),
            trainable=True
        )
        self.b = self.add_weight(
            name='b',
            shape=(1,),
            initializer=tf.constant_initializer(value=initial_b),
            trainable=True
        )
        
    def call(self, inputs):
        
        a_layer = tf.where(inputs > self.a, 1.0, inputs / self.a)
        b_layer = tf.clip_by_value(tf.where(inputs > self.b, 1.0, (inputs - self.a) / (self.b - self.a)), 0, 1)
        c_layer = tf.clip_by_value((inputs - self.b) / (1 - self.b), 0, 1)


        return a_layer, b_layer, c_layer




def define_model(initial_a, initial_b):
    input_shape = (32, 32, 32, 1)
    input_data = Input(shape=input_shape)

    # Use the custom layer with trainable variables
    dynamic_range_layer = LearnableDynamicRange(initial_a=initial_a, initial_b=initial_b)(input_data)
    a_layer, b_layer, c_layer = dynamic_range_layer

    branch_a = conv_block(a_layer)
    branch_b = conv_block(b_layer)
    branch_c = conv_block(c_layer)

    

    concatenated_branches = concatenate([branch_a, branch_b, branch_c])

    x = Dense(128, activation='relu', kernel_initializer='he_uniform')(concatenated_branches)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(8, activation='relu', kernel_initializer='he_uniform')(x)

    output_layer = Dense(1, activation='sigmoid', kernel_initializer='he_uniform')(x)

    model = Model(inputs=input_data, outputs=output_layer)

    return model

In [ ]:
with strategy.scope():
    model=define_model(0.33,0.66)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)
checkpoint_callback = ModelCheckpoint('Luna_train4.h5', monitor='val_accuracy',save_best_only=True, save_format='tf')

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync


history = model.fit(X_train, y_train, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[checkpoint_callback,lr_callback],verbose=0)
    

best_model4 = tf.keras.models.load_model('Luna_train4.h5', custom_objects={'LearnableDynamicRange': LearnableDynamicRange})

loss4, accuracy4 = best_model4.evaluate(X_test, y_test)
print('accuracy: ',accuracy4*100)
print('loss: ',loss4*100)

print('--------------------------------------------------------------------')
a_value = best_model4.get_layer('learnable_dynamic_range').a.numpy()
b_value = best_model4.get_layer('learnable_dynamic_range').b.numpy()
print("Value of 'a' in the best model:", a_value)
print("Value of 'b' in the best model:", b_value)

In [ ]:
luna_correct_hu4=[]
predictions4 = best_model4.predict(X_test)
predicted_classes = (predictions4 > 0.5).astype(int)

#---------------------------------------------------------------------------------
# Generate the classification report
class_names = list(['benign','malignant'])
report = classification_report(y_test, predicted_classes, target_names=class_names,digits=5)
print("Classification Report:\n", report)

# Calculate AUC
auc = roc_auc_score(y_test, predictions4)
print(f'AUC: {auc}')



# Generate the confusion matrix
cm = confusion_matrix(y_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# Extract micro avg 
report0 = classification_report(y_test, predicted_classes, target_names=class_names,digits=5,output_dict=True)
precision = report0['macro avg']['precision']
recall = report0['macro avg']['recall']
f1_score = report0['macro avg']['f1-score']
combined_row = {'AUC': auc*100, 'Precision': precision*100, 'Recall': recall*100, 'F1-Score': f1_score*100}
combined_row_df = pd.DataFrame([combined_row])
auc_df = pd.concat([auc_df, combined_row_df], ignore_index=True)

# HU (with original input)
## Dividing into 2 branches constant initializer

In [ ]:
# Custom layer with trainable variables
class LearnableDynamicRange(Layer):
    def __init__(self, initial_a, **kwargs):
        # Use the provided name or the default name if not provided
        name = kwargs.pop('name', 'learnable_dynamic_range')
        super(LearnableDynamicRange, self).__init__(name=name, **kwargs)

        # Define trainable variables with initial values
        self.a = self.add_weight(
            name='a',
            shape=(1,),
            initializer=tf.constant_initializer(value=initial_a),
            trainable=True
        )
        
        
    def call(self, inputs):
        
        a_layer = tf.where(inputs > self.a, 1.0, inputs / self.a)
        b_layer = tf.clip_by_value((inputs - self.a) / (1 - self.a), 0, 1)
        c_layer = inputs

        return a_layer, b_layer, c_layer



def define_model(initial_a):
    input_shape = (32, 32, 32, 1)
    input_data = Input(shape=input_shape)

    # Use the custom layer with trainable variables
    dynamic_range_layer = LearnableDynamicRange(initial_a=initial_a)(input_data)
    a_layer, b_layer, c_layer = dynamic_range_layer

    branch_a = conv_block(a_layer)
    branch_b = conv_block(b_layer)
    branch_c = conv_block(c_layer)
    

    concatenated_branches = concatenate([branch_a, branch_b, branch_c])

    x = Dense(128, activation='relu', kernel_initializer='he_uniform')(concatenated_branches)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(8, activation='relu', kernel_initializer='he_uniform')(x)

    output_layer = Dense(1, activation='sigmoid', kernel_initializer='he_uniform')(x)

    model = Model(inputs=input_data, outputs=output_layer)

    return model

In [ ]:
with strategy.scope():
    model=define_model(0.5)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)
checkpoint_callback = ModelCheckpoint('Luna_train14.h5', monitor='val_accuracy',save_best_only=True, save_format='tf')

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync


history = model.fit(X_train, y_train, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[checkpoint_callback,lr_callback],verbose=0)
    

best_model14 = tf.keras.models.load_model('Luna_train14.h5', custom_objects={'LearnableDynamicRange': LearnableDynamicRange})

loss14, accuracy14 = best_model14.evaluate(X_test, y_test)
print('accuracy: ',accuracy14*100)
print('loss: ',loss14*100)

print('------------------------------------------------')
a_value = best_model14.get_layer('learnable_dynamic_range').a.numpy()
print("Value of 'a' in the best model:", a_value)

In [ ]:
luna_correct_hu14=[]
predictions14 = best_model14.predict(X_test)
predicted_classes = (predictions14 > 0.5).astype(int)


#---------------------------------------------------------------------------------
# Generate the classification report
class_names = list(['benign','malignant'])
report = classification_report(y_test, predicted_classes, target_names=class_names,digits=5)
print("Classification Report:\n", report)

# Calculate AUC
auc = roc_auc_score(y_test, predictions14)
print(f'AUC: {auc}')



# Generate the confusion matrix
cm = confusion_matrix(y_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# Extract micro avg 
report0 = classification_report(y_test, predicted_classes, target_names=class_names,digits=5,output_dict=True)
precision = report0['macro avg']['precision']
recall = report0['macro avg']['recall']
f1_score = report0['macro avg']['f1-score']
combined_row = {'AUC': auc*100, 'Precision': precision*100, 'Recall': recall*100, 'F1-Score': f1_score*100}
combined_row_df = pd.DataFrame([combined_row])
auc_df = pd.concat([auc_df, combined_row_df], ignore_index=True)

# Dividing into 3 branches constant initializer

In [ ]:
# Custom layer with trainable variables
class LearnableDynamicRange(Layer):
    def __init__(self, initial_a, initial_b, **kwargs):
        # Use the provided name or the default name if not provided
        name = kwargs.pop('name', 'learnable_dynamic_range')
        super(LearnableDynamicRange, self).__init__(name=name, **kwargs)

        # Define trainable variables with initial values
        self.a = self.add_weight(
            name='a',
            shape=(1,),
            initializer=tf.constant_initializer(value=initial_a),
            trainable=True
        )
        self.b = self.add_weight(
            name='b',
            shape=(1,),
            initializer=tf.constant_initializer(value=initial_b),
            trainable=True
        )
        
    def call(self, inputs):
        
        a_layer = tf.where(inputs > self.a, 1.0, inputs / self.a)
        b_layer = tf.clip_by_value(tf.where(inputs > self.b, 1.0, (inputs - self.a) / (self.b - self.a)), 0, 1)
        c_layer = tf.clip_by_value((inputs - self.b) / (1 - self.b), 0, 1)
        d_layer = inputs

        return a_layer, b_layer, c_layer, d_layer



def define_model(initial_a, initial_b):
    input_shape = (32, 32, 32, 1)
    input_data = Input(shape=input_shape)

    # Use the custom layer with trainable variables
    dynamic_range_layer = LearnableDynamicRange(initial_a=initial_a, initial_b=initial_b)(input_data)
    a_layer, b_layer, c_layer, d_layer = dynamic_range_layer

    branch_a = conv_block(a_layer)
    branch_b = conv_block(b_layer)
    branch_c = conv_block(c_layer)
    branch_d = conv_block(d_layer)
    

    concatenated_branches = concatenate([branch_a, branch_b, branch_c,branch_d])

    x = Dense(128, activation='relu', kernel_initializer='he_uniform')(concatenated_branches)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dense(8, activation='relu', kernel_initializer='he_uniform')(x)

    output_layer = Dense(1, activation='sigmoid', kernel_initializer='he_uniform')(x)

    model = Model(inputs=input_data, outputs=output_layer)

    return model

In [ ]:
with strategy.scope():
    model=define_model(0.33,0.66)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.001,
        patience=5,
        min_lr=1e-6)
checkpoint_callback = ModelCheckpoint('Luna_train16.h5', monitor='val_accuracy',save_best_only=True, save_format='tf')

BATCH_SIZE_TPU = BS * strategy.num_replicas_in_sync


history = model.fit(X_train, y_train, validation_data=(X_val1, y_val1), epochs=E, batch_size=BATCH_SIZE_TPU,callbacks=[checkpoint_callback,lr_callback],verbose=0)
    

best_model16 = tf.keras.models.load_model('Luna_train16.h5', custom_objects={'LearnableDynamicRange': LearnableDynamicRange})

loss16, accuracy16 = best_model16.evaluate(X_test, y_test)
print('accuracy: ',accuracy16*100)
print('loss: ',loss16*100)


print('-----------------------------------------------------------------')
a_value = best_model16.get_layer('learnable_dynamic_range').a.numpy()
b_value = best_model16.get_layer('learnable_dynamic_range').b.numpy()
 
print("Value of 'a' in the best model:", a_value)
print("Value of 'b' in the best model:", b_value)

In [ ]:
luna_correct_hu16=[]
predictions16 = best_model16.predict(X_test)
predicted_classes = (predictions16 > 0.5).astype(int)


#---------------------------------------------------------------------------------
# Generate the classification report
class_names = list(['benign','malignant'])
report = classification_report(y_test, predicted_classes, target_names=class_names,digits=5)
print("Classification Report:\n", report)

# Calculate AUC
auc = roc_auc_score(y_test, predictions16)
print(f'AUC: {auc}')



# Generate the confusion matrix
cm = confusion_matrix(y_test, predicted_classes )
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# Extract micro avg 
report0 = classification_report(y_test, predicted_classes, target_names=class_names,digits=5,output_dict=True)
precision = report0['macro avg']['precision']
recall = report0['macro avg']['recall']
f1_score = report0['macro avg']['f1-score']
combined_row = {'AUC': auc*100, 'Precision': precision*100, 'Recall': recall*100, 'F1-Score': f1_score*100}
combined_row_df = pd.DataFrame([combined_row])
auc_df = pd.concat([auc_df, combined_row_df], ignore_index=True)

In [ ]:
accuracy_list = [round(accuracy * 100, 2) for accuracy in [accuracy1, accuracy2, accuracy4, accuracy14, accuracy16]]

col_head = [
    '2 ranges (without original input, random)', 
    '2 ranges (without original input, constant)', 
    '3 ranges (without original input, constant)',
    '2 ranges (+ original input, constant)',
    '3 ranges (+ original input, constant)'
]

df_acc = pd.DataFrame({'type': col_head, 'accuracy': accuracy_list})
print(df_acc)
print(auc_df)
# Save Excel file
df_acc.to_excel('df_acc.xlsx', index=False)
auc_df.to_excel('df_auc.xlsx', index=False)